In [5]:
import pandas as pd
from os.path import basename
from os.path import dirname
pd.set_option('display.max_colwidth', 200)

In [6]:
!find /ccmb/BioinfCore/DNASeqCore/Run_163* -maxdepth 5 -name '*.fastq.gz' | grep -v 'Undetermined' > /tmp/fastqs.tmp
!wc -l /tmp/fastqs.tmp
fastqs_df = pd.read_table('/tmp/fastqs.tmp', names=['path'])
print(fastqs_df.shape)
fastqs_df.head()

1789 /tmp/fastqs.tmp
(1789, 1)


,path
0,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L002_R1_001.fastq.gz
1,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L001_R2_001.fastq.gz
2,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L002_R2_001.fastq.gz
3,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L001_R1_001.fastq.gz
4,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64540/64540_TAGCTT_S31_L003_R1_001.fastq.gz


In [8]:
fastqs_df['dirname'] = fastqs_df.path.apply(dirname)
fastqs_df['basename'] = fastqs_df.path.apply(basename)
fastqs_df['prefix'] = fastqs_df.basename.str.split('_R[12][_\.]').apply(lambda x:x[0])
fastqs_df['read'] = fastqs_df.basename.apply(lambda x: 'r1' if '_R1' in x else 'r2' if '_R2' in x else 'rx')
fastqs_df.head()

,path,dirname,basename,prefix,read
0,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L002_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555,64555_GGCTAC_S10_L002_R1_001.fastq.gz,64555_GGCTAC_S10_L002,r1
1,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L001_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555,64555_GGCTAC_S10_L001_R2_001.fastq.gz,64555_GGCTAC_S10_L001,r2
2,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L002_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555,64555_GGCTAC_S10_L002_R2_001.fastq.gz,64555_GGCTAC_S10_L002,r2
3,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555/64555_GGCTAC_S10_L001_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64555,64555_GGCTAC_S10_L001_R1_001.fastq.gz,64555_GGCTAC_S10_L001,r1
4,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64540/64540_TAGCTT_S31_L003_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64540,64540_TAGCTT_S31_L003_R1_001.fastq.gz,64540_TAGCTT_S31_L003,r1


In [9]:
fastqs_df['read'].value_counts()

r1    1002
r2     787
Name: read, dtype: int64

In [10]:
fastqs_df[fastqs_df['read']=='rx']

,path,dirname,basename,prefix,read


In [11]:
fastq_by_read_df = pd.pivot_table(fastqs_df[['dirname','prefix','basename','read']],
                                  index=['dirname', 'prefix'],
                                  values='basename',
                                  columns='read',
                                  aggfunc='min')
fastq_by_read_df.head()

read                                                                                                                           r1  \
dirname                                                              prefix                                                         
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528 64528_ATTCCT_S19_L003  64528_ATTCCT_S19_L003_R1_001.fastq.gz   
                                                                     64528_ATTCCT_S19_L004  64528_ATTCCT_S19_L004_R1_001.fastq.gz   
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529 64529_GGCTAC_S20_L003  64529_GGCTAC_S20_L003_R1_001.fastq.gz   
                                                                     64529_GGCTAC_S20_L004  64529_GGCTAC_S20_L004_R1_001.fastq.gz   
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530 64530_ACTTGA_S21_L003  64530_ACTTGA_S21_L003_R1_001.fastq.gz   

read                                                                                                                           r2  
dirname                                                              prefix                                                        
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528 64528_ATTCCT_S19_L003  64528_ATTCCT_S19_L003_R2_001.fastq.gz  
                                                                     64528_ATTCCT_S19_L004  64528_ATTCCT_S19_L004_R2_001.fastq.gz  
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529 64529_GGCTAC_S20_L003  64529_GGCTAC_S20_L003_R2_001.fastq.gz  
                                                                     64529_GGCTAC_S20_L004  64529_GGCTAC_S20_L004_R2_001.fastq.gz  
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530 64530_ACTTGA_S21_L003  64530_ACTTGA_S21_L003_R2_001.fastq.gz

In [13]:
fastq_r12_df = fastq_by_read_df[(~fastq_by_read_df.r1.isnull()) & (~fastq_by_read_df.r2.isnull())].reset_index()
fastq_r12_df['r1_path'] = fastq_r12_df.dirname + '/' + fastq_r12_df.r1
fastq_r12_df['r2_path'] = fastq_r12_df.dirname + '/' + fastq_r12_df.r2
fastq_r12_df.head()

read,dirname,prefix,r1,r2,r1_path,r2_path
0,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528,64528_ATTCCT_S19_L003,64528_ATTCCT_S19_L003_R1_001.fastq.gz,64528_ATTCCT_S19_L003_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L003_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L003_R2_001.fastq.gz
1,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528,64528_ATTCCT_S19_L004,64528_ATTCCT_S19_L004_R1_001.fastq.gz,64528_ATTCCT_S19_L004_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L004_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L004_R2_001.fastq.gz
2,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529,64529_GGCTAC_S20_L003,64529_GGCTAC_S20_L003_R1_001.fastq.gz,64529_GGCTAC_S20_L003_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L003_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L003_R2_001.fastq.gz
3,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529,64529_GGCTAC_S20_L004,64529_GGCTAC_S20_L004_R1_001.fastq.gz,64529_GGCTAC_S20_L004_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L004_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L004_R2_001.fastq.gz
4,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530,64530_ACTTGA_S21_L003,64530_ACTTGA_S21_L003_R1_001.fastq.gz,64530_ACTTGA_S21_L003_R2_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530/64530_ACTTGA_S21_L003_R1_001.fastq.gz,/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530/64530_ACTTGA_S21_L003_R2_001.fastq.gz


In [14]:
fastq_r12_df[['r1_path', 'r2_path']].to_csv('/tmp/fastq_r12.tmp', sep=' ', index=False)
!head /tmp/fastq_r12.tmp
!wc -l /tmp/fastq_r12.tmp

r1_path r2_path
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L003_R1_001.fastq.gz /ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L003_R2_001.fastq.gz
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L004_R1_001.fastq.gz /ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64528/64528_ATTCCT_S19_L004_R2_001.fastq.gz
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L003_R1_001.fastq.gz /ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L003_R2_001.fastq.gz
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L004_R1_001.fastq.gz /ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64529/64529_GGCTAC_S20_L004_R2_001.fastq.gz
/ccmb/BioinfCore/DNASeqCore/Run_1630/rhim/Run_1630/rhim/Sample_64530/64530_ACTTGA_S21_L003_R1_001.fastq.gz /ccmb/BioinfCore/DNAS

In [21]:
!(tail -n +2 /tmp/fastq_r12.tmp | xargs -P 4 -L 1 diff -s ) 2>&1 > /tmp/fastq_r12.identical.tmp

In [22]:
!wc -l /tmp/fastq_r12.identical.tmp

787 /tmp/fastq_r12.identical.tmp


In [23]:
!grep identical /tmp/fastq_r12.identical.tmp